# Recursos computacionais da GAN quântica

Este notebook replica a análise de custo computacional realizada para as GANs clássicas, agora aplicada ao gerador quântico utilizado nos experimentos com o BreastMNIST. A proposta é medir tempo de treinamento por classe, tamanho do gerador e tempo médio de inferência de uma imagem sintética.


In [8]:
import time
from statistics import mean

import pandas as pd
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import transforms

from medmnist_data import load_medmnist_data
from quantum_gan_medmnist import PatchQuantumGenerator, Discriminator, train_quantum_gan


In [9]:
DATA_FLAG = "breastmnist"
BATCH_SIZE = 128
NUM_EPOCHS = 50

N_QUBITS = 5
N_A_QUBITS = 1
TARGET_IMG_SIZE = 8
Q_DEPTH = 6

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device_str = "cuda" if torch.cuda.is_available() else "cpu"

transform_lowres = transforms.Compose([
    transforms.Resize((TARGET_IMG_SIZE, TARGET_IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
])

bundle = load_medmnist_data(
    data_flag=DATA_FLAG,
    batch_size=BATCH_SIZE,
    download=True,
    transform=transform_lowres,
    shuffle_train=True,
)

train_dataset = bundle.train_dataset
label_names = {int(k): v for k, v in bundle.label_names.items()}
label_ids = sorted(label_names.keys())

patch_size = 2 ** (N_QUBITS - N_A_QUBITS)
if (TARGET_IMG_SIZE ** 2) % patch_size != 0:
    raise ValueError("target_img_size**2 deve ser múltiplo de patch_size para montar a imagem completa")
N_GENERATORS = (TARGET_IMG_SIZE ** 2) // patch_size
LATENT_DIM = N_QUBITS

def subset_by_label(dataset, label):
    indices = [i for i in range(len(dataset)) if int(dataset.labels[i]) == label]
    return Subset(dataset, indices)

train_loaders = {
    label: DataLoader(
        subset_by_label(train_dataset, label),
        batch_size=BATCH_SIZE,
        shuffle=True,
    )
    for label in label_ids
}


Using downloaded and verified file: /home/mahlow/.medmnist/breastmnist.npz
Using downloaded and verified file: /home/mahlow/.medmnist/breastmnist.npz


/tmp/ipykernel_549363/508418954.py:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  indices = [i for i in range(len(dataset)) if int(dataset.labels[i]) == label]


In [10]:
def count_parameters(model):
    return sum(param.numel() for param in model.parameters())

def measure_inference_time(generator, *, latent_dim, num_runs=32):
    generator.eval()
    sync = torch.cuda.synchronize if torch.cuda.is_available() else (lambda: None)
    with torch.no_grad():
        sync()
        start = time.time()
        for _ in range(num_runs):
            noise = torch.rand(1, latent_dim, device=device) * (torch.pi / 2)
            generator(noise)
        sync()
    return (time.time() - start) / num_runs

def measure_inference_time_per_label(generators):
    tempos = []
    rows = []
    for label, generator in generators.items():
        tempo = measure_inference_time(generator, latent_dim=LATENT_DIM)
        tempos.append(tempo)
        rows.append(
            {
                "Label_ID": label,
                "Label_Nome": label_names[label],
                "Tempo_inferência_img_seg": tempo,
            }
        )
    return (mean(tempos) if tempos else float("nan")), rows


In [11]:
def run_quantum_gan():
    generators = {}
    per_label_training = []
    start_total = time.time()

    for label in label_ids:
        generator = PatchQuantumGenerator(
            N_GENERATORS,
            TARGET_IMG_SIZE,
            n_qubits=N_QUBITS,
            n_a_qubits=N_A_QUBITS,
            q_depth=Q_DEPTH,
        ).to(device)
        discriminator = Discriminator(img_size=TARGET_IMG_SIZE).to(device)

        start_label = time.time()
        train_quantum_gan(
            train_loaders[label],
            generator,
            discriminator,
            epochs=NUM_EPOCHS,
            device=device_str,
        )
        elapsed_label = time.time() - start_label

        per_label_training.append(
            {
                "Label_ID": label,
                "Label_Nome": label_names[label],
                "Tempo_treinamento_classe_seg": elapsed_label,
            }
        )
        generators[label] = generator.eval()

    total_time = time.time() - start_total
    avg_inference, per_label_inference = measure_inference_time_per_label(generators)

    summary = {
        "GAN": "QuantumGAN",
        "Tempo_treinamento_seg": total_time,
        "Parametros_Gerador": count_parameters(next(iter(generators.values()))),
        "Tempo_inferência_img_seg": avg_inference,
    }

    return summary, per_label_training, per_label_inference


In [12]:
summary, per_label_training, per_label_inference = run_quantum_gan()

df_summary = pd.DataFrame([summary])
df_summary['Tempo_treinamento_min'] = df_summary['Tempo_treinamento_seg'] / 60
df_summary['Tempo_inferência_img_ms'] = df_summary['Tempo_inferência_img_seg'] * 1_000
df_summary


,GAN,Tempo_treinamento_seg,Parametros_Gerador,Tempo_inferência_img_seg,Tempo_treinamento_min,Tempo_inferência_img_ms
0,QuantumGAN,913.686331,120,0.031649,15.228106,31.648602


In [ ]:
df_treinamento = pd.DataFrame(per_label_training)
df_treinamento['Tempo_treinamento_classe_min'] = df_treinamento['Tempo_treinamento_classe_seg'] / 60
df_treinamento


In [ ]:
df_inferencia = pd.DataFrame(per_label_inference)
df_inferencia['Tempo_inferência_img_ms'] = df_inferencia['Tempo_inferência_img_seg'] * 1_000
df_inferencia
